<a href="https://colab.research.google.com/github/HDWilliams/FashionMNISTClassifier/blob/master/FashionMNISTClassifer_DO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#needed if running on local machine. This project was made using Colab which does not require this step
#!pip install torch torchvision


In [1]:
#import relevant dependancies
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

from torchvision import datasets, transforms

#set up transform for a black and white image i.e. size = [1, #, #]
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

#set up trainset and trainloader and apply transform, do same for testset and testloader
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


0it [00:00, ?it/s]

26427392it [00:04, 5769994.30it/s]                              


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 39078.57it/s]                           
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:02, 1716163.34it/s]                             
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 13477.96it/s]            

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [2]:
#set up GPU training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
#Set up Neural Network
import torch.nn as nn
import torch.nn.functional as F

class FashionClassifier(nn.Module):
  #envoke __init__ method of parent class
  def __init__(self):
    super().__init__()
    
    #define layers of network, 2 hidden layers and one output layer. All layers are linear
    #input of the next layer will be equal to the output of the previous layer as all layers are fully connected
    #output layer must have same number of nodes as classes in the dataset
    
    inp = 784 # 28 X 28 image
    hid1 = 128 
    hid2 = 64
    out = 10 # 10 classes of clothing
    self.layer1 = nn.Linear(inp, hid1)
    self.layer2 = nn.Linear(hid1, hid2)
    self.outputLayer = nn.Linear(hid2, out)
    
    #drop out to prevent overfitting
    self.Dropout = nn.Dropout(p=.2)
  
  def forward(self, x):
    x = self.Dropout(F.relu(self.layer1(x)))
    x = self.Dropout(F.relu(self.layer2(x)))
    
    log_softmax = nn.LogSoftmax(dim=1)
    output = log_softmax(self.outputLayer(x))
    return output

model = FashionClassifier()
model.to(device)


FashionClassifier(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (outputLayer): Linear(in_features=64, out_features=10, bias=True)
  (Dropout): Dropout(p=0.2)
)

In [0]:
#set up optimizer for SGD and criterion for loss function
#loss function will be Negative Log Likelihood for multiclass classification 

#learning rate = lr
lr = .05

criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [7]:
#train the model
epochs = 25
loss_by_epoch = []
for epoch in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    #reshape images into expected 2D
    images = images.view(images.shape[0], -1)
    optimizer.zero_grad()
    
    #send images and labels to device for potential GPU training if available 
    images, labels = images.to(device), labels.to(device)
    
    
    output = model(images)
    loss = criterion(output, labels)
    
    #calculate gradients and backpropagate 
    loss.backward()
    
    #take step using SGD
    optimizer.step()
    
    running_loss += loss.item()
  #print loss every epoch
  loss_by_epoch.append(running_loss)
  print ('Loss: ', running_loss / len(trainloader))
print(loss_by_epoch)
    
    
    
  

Loss:  0.2913553770909558
Loss:  0.2821006122380813
Loss:  0.2792581195500233
Loss:  0.27412770238162865
Loss:  0.2722510958372403
Loss:  0.26761628470536486
Loss:  0.26435852032512236
Loss:  0.26094895795082995
Loss:  0.25889775651826785
Loss:  0.25311797530825203
Loss:  0.25209863379851843
Loss:  0.24985804420703256
Loss:  0.24751620506967054
Loss:  0.24294868668418196
Loss:  0.24018737869158482
Loss:  0.23673706483453322
Loss:  0.23378623303955298
Loss:  0.23434674140137396
Loss:  0.23102239964962767
Loss:  0.22796293984312238
Loss:  0.22852365741692882
Loss:  0.2229721834505799
Loss:  0.21975984314738561
Loss:  0.2196139744095711
Loss:  0.21760240156672148
[273.2913437113166, 264.61037427932024, 261.9441161379218, 257.1317848339677, 255.37152789533138, 251.02407505363226, 247.96829206496477, 244.7701225578785, 242.84609561413527, 237.42466083914042, 236.46851850301027, 234.36684546619654, 232.17020035535097, 227.88586810976267, 225.29576121270657, 222.05936681479216, 219.2914865911

In [0]:
#test Network

#Visualization help functions from GitHub: https://github.com/iArunava/Intro-to-Deep-Learning-with-Pytorch-Udacity-Solutions/blob/master/intro-to-pytorch/Part%204%20-%20Fashion-MNIST.ipynb
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()
    

def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

### end of visulaization helper functions

def viewNumImg(numImgs):
  model.eval()
  data = iter(testloader)
  images, labels = data.next()

  for img in images[0:numImgs]:
    # Convert 2D image to 1D vector
    img = img.view(1, -1)

    # TODO: Calculate the class probabilities (softmax) for img
    ps = torch.exp(model(img))
    print (ps)

    # Plot the image and probabilities
    #helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')
    view_classify(img.resize_(1, 28, 28), ps, version='Fashion')
  model.train()

viewNumImg(3)

In [8]:
#Validation: get validation accuracy on test set
accuracy = 0

with torch.no_grad():
  model.eval()
  for images, labels in testloader:
    images = images.view(images.shape[0], -1)
    
    #send tensors to device
    images, labels = images.to(device), labels.to(device)
    
    ps=torch.exp(model(images))
    topk, top_class = ps.topk(1, dim=1)
    results = top_class == labels.view(top_class.shape[0],1)
    accuracy += torch.mean(results.type(torch.FloatTensor))
print('Accuracy: ', (accuracy.item()/len(testloader))*100, '%')
model.train()

Accuracy:  87.36066878980891 %


FashionClassifier(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (outputLayer): Linear(in_features=64, out_features=10, bias=True)
  (Dropout): Dropout(p=0.2)
)